<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas/pandas_medium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

176. Second Highest Salary

https://leetcode.com/problems/second-highest-salary/description/

```sql
SELECT max(salary) as SecondHighestSalary
FROM (
SELECT
    id, salary,
    dense_rank() OVER (ORDER BY salary DESC) AS rn
FROM Employee e)
WHERE rn = 2
```

In [ ]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})

In [ ]:
def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    result = employee.assign(rn = employee.salary.rank(method='dense', ascending=False)).query("rn == 2")[['salary']].rename({"salary":"SecondHighestSalary"}, axis=1)
    return result.head(1) if result.shape[0] else pd.DataFrame([{"SecondHighestSalary": None}])

177. Nth Highest Salary

https://leetcode.com/problems/nth-highest-salary/description/

```sql
CREATE OR REPLACE FUNCTION NthHighestSalary(N INT) RETURNS TABLE (Salary INT) AS $$
BEGIN
  RETURN QUERY (
    -- Write your PostgreSQL query statement below.
    SELECT max(sub.salary) FROM (
    SELECT
        e.salary,
        dense_rank() OVER (ORDER BY e.salary DESC) as rn
    FROM Employee e) sub
    WHERE sub.rn = N
      
  );
END;
$$ LANGUAGE plpgsql;
```

In [ ]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['Id', 'Salary']).astype({'Id':'Int64', 'Salary':'Int64'})

In [ ]:
def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    result = employee.assign(rn = employee.salary.rank(method='dense', ascending=False)).query("rn == @N")[['salary']].rename({"salary":f"getNthHighestSalary({N})"}, axis=1)
    return result.head(1) if result.shape[0] else pd.DataFrame([{f"getNthHighestSalary({N})": None}])

178. Rank Scores

https://leetcode.com/problems/rank-scores/description/

```sql
SELECT
    score,
    dense_rank() OVER (ORDER BY score DESC) as rank
FROM Scores
ORDER BY score DESC
```

In [ ]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})

In [ ]:
def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    return scores.assign(rank = scores.score.rank(method='dense', ascending=False))[['score','rank']].sort_values(by='score', ascending=False)

180. Consecutive Numbers

https://leetcode.com/problems/consecutive-numbers/

```sql
-- Write your PostgreSQL query statement below
SELECT
    DISTINCT consecutivenums FROM (
SELECT
    num as ConsecutiveNums,
    lag(num) OVER (ORDER BY id) as b,
    lead(num) OVER (ORDER BY id) as a
 FROM Logs) sub
 WHERE ConsecutiveNums = a AND ConsecutiveNums = b
 ```


In [2]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})

In [17]:
def consecutive_numbers(logs: pd.DataFrame) -> pd.DataFrame:
    condition = ((logs.num.shift(1) == logs.num) & ((logs.num.shift(-1) == logs.num)))
    return logs[condition]['num'].drop_duplicates().rename('ConsecutiveNums').to_frame()

184. Department Highest Salary

https://leetcode.com/problems/department-highest-salary/description/

```sql
SELECT
    d.name AS Department,
    sub.Employee,
    sub.Salary
FROM (
SELECT
    e.departmentId,
    e.name AS Employee ,
    salary AS Salary,
    RANK() OVER (PARTITION BY departmentId ORDER BY Salary DESC) as rnk
FROM Employee e) sub
LEFT JOIN Department d ON sub.departmentId = d.id
WHERE sub.rnk = 1
```

In [38]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [51]:
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    return employee[employee.groupby('departmentId')['salary']\
                    .rank(method='min', ascending=False) == 1].merge(department, how='left', left_on='departmentId', right_on='id')\
                    .rename({"name_y": "Department", "name_x": "Employee", "salary": "Salary"}, axis=1)[["Department", "Employee", "Salary"]]

550. Game Play Analysis IV

https://leetcode.com/problems/game-play-analysis-iv/

```sql
-- Write your PostgreSQL query statement below
WITH t AS (
SELECT
    a.player_id,
    min(a.event_date) + INTERVAL '1 day ' as next_day
FROM Activity a
GROUP BY a.player_id )

SELECT
ROUND(COUNT(a.player_id)*1.0 / (SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction  
FROM Activity a
JOIN t ON a.player_id = t.player_id AND t.next_day = a.event_date

```

In [53]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})

In [98]:
def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    next_day = activity.groupby("player_id").event_date.apply(lambda x: x.min() + pd.DateOffset(days=1))
    fraction = round(activity.assign(next_day=activity.player_id.map(next_day)).query("event_date == next_day").player_id.nunique() / activity.player_id.nunique(), 2)
    return pd.DataFrame([fraction], columns=['fraction'], index=[0])

570. Managers with at Least 5 Direct Reports

https://leetcode.com/problems/managers-with-at-least-5-direct-reports/description/

```sql
SELECT
    e1.name
FROM Employee e1
JOIN Employee e2 ON e1.id = e2.managerId
GROUP BY e1.id, e1.name
HAVING count(e2.managerId) >= 5
```

In [116]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})

In [127]:
def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    filtered = employee.groupby('managerId', ).agg(reporting = ('id', 'count')).query('reporting >=5', engine='python')
    return employee[employee.id.isin(filtered.index)][['name']]

585. Investments in 2016

https://leetcode.com/problems/investments-in-2016/description/

```sql
WITH uniqu_ltt AS(
SELECT
    lat::text||' '||lon::text as ltconc
FROM Insurance i
GROUP BY lat::text||' '||lon::text
HAVING count(*) = 1),

repeted_tiv AS (
SELECT
tiv_2015
FROM Insurance i
GROUP BY tiv_2015
HAVING COUNT(tiv_2015) > 1)

SELECT
ROUND(sum(tiv_2016)::NUMERIC, 2) as tiv_2016
FROM Insurance
WHERE tiv_2015 in (SELECT tiv_2015 FROM repeted_tiv) AND lat::text||' '||lon::text IN (SELECT ltconc FROM uniqu_ltt)
```

In [149]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})

In [151]:
def find_investments(insurance: pd.DataFrame) -> pd.DataFrame:
    uniqu_ltt  = insurance.assign(ltconc = insurance.lat.astype(str).str.cat(insurance.lon.astype(str), sep=' ')).groupby('ltconc').agg(ltconc=('pid', 'count')).query("ltconc == 1", engine='python').index
    repeted_tiv  = insurance.groupby('tiv_2015').agg(tiv_2015_cnt = ('tiv_2015', 'count')).query("tiv_2015_cnt > 1", engine='python').index
    tiv_2016 = insurance[insurance.tiv_2015.isin(repeted_tiv) & insurance.lat.astype(str).str.cat(insurance.lon.astype(str), sep=' ').isin(uniqu_ltt)].tiv_2016.sum().round(2)
    return pd.DataFrame([tiv_2016], columns=['tiv_2016'], index=[0])

602. Friend Requests II: Who Has the Most Friends

https://leetcode.com/problems/friend-requests-ii-who-has-the-most-friends/description/

```sql
WITH temp as
(SELECT requester_id as id FROM RequestAccepted
UNION ALL
SELECT accepter_id as id FROM RequestAccepted)

SELECT id, COUNT(id) as num FROM temp GROUP BY id
ORDER BY num DESC LIMIT 1
```

In [157]:
data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype({'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'})

In [231]:
def most_friends(request_accepted: pd.DataFrame) -> pd.DataFrame:
    result = request_accepted.requester_id.to_list() + request_accepted.accepter_id.to_list()
    return pd.DataFrame(result, columns=['id'], index=range(len(result))).value_counts().sort_values(ascending=False).head(1).reset_index(name='num')

In [245]:
import pandas as pd
from collections import Counter

def most_friends(request_accepted: pd.DataFrame) -> pd.DataFrame:
    c = Counter(request_accepted.requester_id)
    c.update(request_accepted.accepter_id)
    return pd.DataFrame(c.most_common()[:1], columns=['id', 'num'], index=[0])

608. Tree Node

https://leetcode.com/problems/tree-node/description/

```sql
SELECT
    id,
    CASE WHEN p_id IS NULL THEN 'Root'
         WHEN id in (SELECT p_id FROM Tree) THEN 'Inner'
         ELSE 'Leaf' END as "type"
FROM Tree
```

In [246]:
data = [[1, None], [2, 1], [3, 1], [4, 2], [5, 2]]
tree = pd.DataFrame(data, columns=['id', 'p_id']).astype({'id':'Int64', 'p_id':'Int64'})

In [264]:
import numpy as np

def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
    tree["type"] = np.where(tree.p_id.isna(), 'Root',
                        np.where(~tree.id.isin(tree.p_id), 'Leaf', 'Inner'))
    return tree[['id', 'type']]

626. Exchange Seats

https://leetcode.com/problems/exchange-seats/description/

```sql
SELECT
id,
CASE WHEN id % 2 = 1 THEN LEAD(student, 1, student) OVER (ORDER BY id)
    ELSE LAG(student, 1, student) OVER (ORDER BY id) END AS student
FROM Seat
```

In [307]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})

In [324]:
def exchange_seats(seat: pd.DataFrame) -> pd.DataFrame:
    seat['student'] = np.where((seat.id %2 == 1) & (seat.id != seat.shape[0]), seat.student.shift(-1),
                               np.where(seat.id %2 == 0, seat.student.shift(1), seat.student))
    return seat

1045. Customers Who Bought All Products

https://leetcode.com/problems/customers-who-bought-all-products/description/

```sql
SELECT
    customer_id
FROM Customer
GROUP BY customer_id
HAVING ARRAY_LENGTH(ARRAY_AGG(DISTINCT product_key), 1) = (SELECT COUNT(*) FROM Product);

```

In [325]:
data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype({'customer_id':'Int64', 'product_key':'Int64'})
data = [[5], [6]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key':'Int64'})

In [355]:
def find_customers(customer: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    all_product = set(product.product_key.values)
    return customer.groupby('customer_id')\
                   .product_key\
                   .apply(lambda x: set(x))\
                   .reset_index(name='product_keys')\
                   .query('product_keys == @all_product')\
                   [['customer_id']]

1070. Product Sales Analysis III

https://leetcode.com/problems/product-sales-analysis-iii/description/

```sql
-- Write your PostgreSQL query statement below
WITH t as (
SELECT
    product_id,
    min(year) as min_year
FROM Sales
GROUP BY product_id)

SELECT
    s.product_id,
    s.year as first_year ,
    s.quantity,
    s.price
FROM Sales s
JOIN t ON t.product_id = s.product_id AND t.min_year = s.year
```

In [356]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})

In [358]:
def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    return sales.assign(min_year = sales.groupby('product_id').year.rank(method='min'))\
                .query('min_year ==  1', engine='python')\
                .iloc[:,1:-1]\
                .rename({'year': 'first_year'}, axis=1)

1158. Market Analysis I

https://leetcode.com/problems/market-analysis-i/description/

```sql
SELECT
    u.user_id as buyer_id,
    u.join_date,
    count(o.order_id) as orders_in_2019
FROM Users u
LEFT JOIN Orders o ON u.user_id = o.buyer_id AND date_part('year', o.order_date) = 2019
GROUP BY u.user_id, u.join_date
```

In [370]:
data = [[1, '2018-01-01', 'Lenovo'], [2, '2018-02-09', 'Samsung'], [3, '2018-01-19', 'LG'], [4, '2018-05-21', 'HP']]
users = pd.DataFrame(data, columns=['user_id', 'join_date', 'favorite_brand']).astype({'user_id':'Int64', 'join_date':'datetime64[ns]', 'favorite_brand':'object'})
data = [[1, '2019-08-01', 4, 1, 2], [2, '2018-08-02', 2, 1, 3], [3, '2019-08-03', 3, 2, 3], [4, '2018-08-04', 1, 4, 2], [5, '2018-08-04', 1, 3, 4], [6, '2019-08-05', 2, 2, 4]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'item_id', 'buyer_id', 'seller_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'Int64', 'buyer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Samsung'], [2, 'Lenovo'], [3, 'LG'], [4, 'HP']]
items = pd.DataFrame(data, columns=['item_id', 'item_brand']).astype({'item_id':'Int64', 'item_brand':'object'})

In [384]:
def market_analysis(users: pd.DataFrame, orders: pd.DataFrame, items: pd.DataFrame) -> pd.DataFrame:
    buyers = orders[orders.order_date.dt.year == 2019]\
                  .groupby('buyer_id')\
                  .order_id.count()\
                  .reset_index(name='orders_in_2019')
    return users.merge(buyers, how='left', left_on='user_id', right_on='buyer_id')[['user_id', 'join_date', 'orders_in_2019']].fillna({'orders_in_2019': 0})\
                            .rename({'user_id': 'buyer_id'}, axis=1)

1164. Product Price at a Given Date

https://leetcode.com/problems/product-price-at-a-given-date/description/

```sql
WITH t as (
SELECT
    s1.product_id,
    s2.new_price,
    row_number() OVER (PARTITION BY s1.product_id ORDER BY s2.change_date DESC NULLS LAST) as rn
FROM Products s1
LEFT JOIN  Products s2 ON s1.product_id = s2.product_id AND s1.change_date = s2.change_date  AND s2.change_date <= '2019-08-16')

SELECT
    product_id,
    COALESCE(new_price, 10) AS  price
FROM t
WHERE rn = 1
```

In [385]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})

In [414]:
def price_at_given_date(products: pd.DataFrame) -> pd.DataFrame:
    temp = products[products.change_date.le('2019-08-16')]\
      .assign(max_date = products[products.change_date.le('2019-08-16')]\
              .groupby('product_id')\
              .change_date.rank(method='min', ascending=False))\
              .query("max_date == 1")\
              .set_index('product_id')\
              .new_price
    products['price'] = products.product_id\
                                    .map(temp)\
                                    .fillna(10)
    return products[['product_id','price']].drop_duplicates()

1174. Immediate Food Delivery II

https://leetcode.com/problems/immediate-food-delivery-ii/description/

```sql

WITH t as (
SELECT
    *,
    row_number() OVER (PARTITION BY customer_id ORDER BY order_date) as rn
FROM Delivery)


SELECT
    round(count(delivery_id) FILTER (WHERE order_date = customer_pref_delivery_date )*100.0 / count(delivery_id), 2) as immediate_percentage
FROM t
WHERE t.rn = 1
```

In [415]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 2, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-12'], [4, 3, '2019-08-24', '2019-08-24'], [5, 3, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13'], [7, 4, '2019-08-09', '2019-08-09']]
delivery = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({'delivery_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'customer_pref_delivery_date':'datetime64[ns]'})